In [1]:
import src.acquire
import src.prepare
import src.explore
import src.model

In [2]:
path='zillow_df.csv'

df, X_train_explore, \
    X_train_scaled, y_train, \
    X_validate_scaled, y_validate, \
    X_test_scaled, y_test = src.prepare.wrangle_zillow(path)

X_train_explore.shape, X_validate_scaled.shape, X_test_scaled.shape

((11666, 18), (5000, 15), (4167, 15))

In [3]:
df

,Unnamed: 0,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fireplace,garage,hottub_spa,lotsizesquarefeet,poolcnt,roomcnt,yearbuilt,taxvaluedollarcnt,id.1,zip,useid,year,zip_cc,useid_cc,year_cc
0,0,3.0,3.0,1534.0,0.0,0.0,0.0,5000.0,0.0,0.0,1940.0,43439.0,117,96028.0,247.0,1940.0,44,1,52
1,1,2.0,3.0,1316.0,0.0,0.0,0.0,5672.0,0.0,0.0,1923.0,205123.0,1018,95997.0,261.0,1923.0,14,4,35
2,2,2.0,3.0,1458.0,0.0,0.0,0.0,8284.0,1.0,0.0,1970.0,136104.0,1248,97318.0,261.0,1970.0,374,4,82
3,3,2.0,3.0,1766.0,0.0,0.0,0.0,25082.0,0.0,0.0,1980.0,810694.0,1734,96086.0,266.0,1980.0,63,8,92
4,4,1.0,2.0,1421.0,0.0,0.0,0.0,6707.0,0.0,0.0,1911.0,35606.0,1772,96018.0,261.0,1911.0,34,4,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20926,20926,1.0,2.0,994.0,0.0,0.0,0.0,5535.0,0.0,0.0,1950.0,411253.0,62966,96242.0,261.0,1950.0,154,4,62
20927,20927,2.0,3.0,1234.0,1.0,2.0,0.0,3049.0,0.0,5.0,1978.0,363000.0,62967,96351.0,265.0,1978.0,194,7,90
20928,20928,2.5,3.0,1535.0,0.0,2.0,0.0,3500.0,0.0,7.0,1985.0,677171.0,62968,96954.0,261.0,1985.0,284,4,97
20929,20929,2.0,3.0,1969.0,0.0,0.0,0.0,5480.0,1.0,0.0,1950.0,677000.0,62969,96242.0,261.0,1950.0,154,4,62


In [4]:
# import pandas as pd
# df["zip_cc"] = df["zip"].cat.codes
# df

## Notes

1. I already uploaded my env.py file so need need to upload yours.  We only need one to access the data.


2. I already uploaded a .gitignore so also no worries there


3. Data is ready to go, i want to engeneer a feature that combines bed and bath, but get a weird error when I try.    we can trouble shoot after mvp, but code can be found in .prepare file


4. I want to leave 2 categorical variables after transforming them to numeric but I also need help with that
   - "regionidzip"
   - "propertylandusetypeid"
   - They need to be made in to categories before we can do any testing on them (Cat to Numeric)


5. id.1 and Unmaed:0 showd up we also need to delete them